
# Data Preprocessing

* Head orientation coordinate matching in CSV file
* Image sourcing, resize and match with allocated timestamped label
* Eye coordinates using an iris detector

In [1]:
import numpy as np
import pandas as pd
import os, os.path
import cv2
import import_ipynb
#from attentiondetector import *

#We read our labels from two cvs files and do some conversions. The video frames 
#are read from our directory into our data genrators. We create our dataframe and 
#add our columned-labels from a pretrained classifier to create our head orientation labels

#We curate our labels from two cvs files of shape(4395, 4) and (2722, 4) respectively.
data_path01 = "../data/unclean_label/clean_label/clean_label01.csv"
data_path02 = "../data/unclean_label/clean_label/clean_label02.csv"
unclean_label = "../data/unclean_label/unclean_label.csv"

def convert_dtype(path):
    data = pd.read_csv(path)
    df = pd.DataFrame(data)
    df = df.iloc[:,4:8]
    df = df.dropna(axis=0,how='any')
    df = np.array(df,dtype='int64')
    for i in df:
        for j in i:
            j = j.astype('int')
    return df
df1 = convert_dtype(data_path01)
df2 = convert_dtype(data_path02)

print(df1.shape)
print(df2.shape)

(4395, 4)
(2722, 4)


In [2]:
df = np.concatenate([df1,df2])
print(df)

[[1582947215462             0             0             0]
 [1582947215465             0             0             0]
 [1582947215469             0             0             0]
 ...
 [1583028965938           -20             7           -56]
 [1583028965949           -20             7           -56]
 [1583028965959           -20             7           -56]]


In [3]:
from collections import OrderedDict
#We convert our labels to a dictionary with the timestamp as keys and PRY as values

Dict = OrderedDict()
for i in df:
    timestamp = int((i[0])/10)
    Dict[timestamp] = [i[1],i[2],i[3]]

print(len(Dict.keys()))


6885


In [4]:
import cv2
import os,os.path
import numpy as np

v_path = "../data/unclean_data/clean"

#An arrary of timestamps of the first frame of each video is recorded for calculating all frames timestamp
TS = [158294721650,158302893900]

#This function takes a video path and returns an array of timestamp of len(vid_path)
def get_timestamps(vid_path,start_time):
    video = cv2.VideoCapture(vid_path)
    FC = int(video.get(cv2.CAP_PROP_FRAME_COUNT))
    timestamps = []
    count = 0
    while video.isOpened():
        ret,frame = video.read()
        frame = np.array(frame)
        #cv2.imwrite('../data/tf_data/{}.jpg'.format(count), frame)
        FTS = video.get(cv2.CAP_PROP_POS_MSEC)
        timestamp = int(start_time + FTS/10)
        timestamps.append(timestamp)
        count+=1
        if count== FC: break
    return timestamps

data_TS = []

#We call our get_timestamps function on each video in our cwd
count1 = 0
for j in os.listdir(v_path):
    if j.endswith(".mp4"):
        videopath = os.path.join(v_path,j)
        print("This is number:{} video".format(count1+1))
        timestamp_arr = get_timestamps(videopath,TS[count1])
        data_TS.extend(timestamp_arr)
        count1+=1                                                                             
    else:
        continue


data_TS = np.array(data_TS).flatten()
print(len(data_TS))


This is number:1 video


KeyboardInterrupt: 

In [ ]:
import csv

#Here, we match the timestamps of our frames to it's respective PRY(pitch, roll yaw) value

print(len(data_TS))
print(len(Dict))

def get_est(Dict,x):
    for i,j in enumerate(Dict.keys()):
        if j==x:
            break
        elif j<x:
            continue
        else:
            break  
    return j

_match = OrderedDict()

for j in data_TS:
    if j in Dict.keys():
        _match[j] = Dict[j]
    else:
        est = get_est(Dict,j)
        _match[j] = Dict[est]
        continue
        
print(len(_match))

with open('../data/tf_label/labels.csv','w') as w:
    writer = csv.writer(w)
    writer.writerows(_match.values())